In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lmfit.models import GaussianModel,LinearModel

/home/frederik/anaconda3/lib/python3.4/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


#Rutherfordstreuung

##Rutherfordsteuuformel
<br>
$\huge \huge \frac{d\sigma}{d\Omega} = 1.3 \cdot 10^{-3} \left( \frac{Z_1 Z_2}{E_0} \right) \frac{1}{sin^4\left(\frac{\theta}{2}\right)}$

<ul>
    <li>$\sigma$: Differentielle WQ
    <li>$\Theta$: Winkel
    <li>$Z_i$: Kernlagundszahl
    <li>E: Energie
</ul>   

In [2]:
def rutherfordsteuung(theta,Z_1,Z_2,E,b):
    e = 1.602 * 10**-19
    return (Z_1*Z_2*e**2)/(4*E*np.sin((3.14*theta-b)/360)**4)

#Energieverlust von $\alpha$-Strahlung in Luft

# Energie-Kanal kalibration

In [3]:
cd /home/frederik/Dokumente/FP/Rutherford/MessdatenRutherford/kalib/

/home/frederik/Dokumente/FP/Rutherford/MessdatenRutherford/kalib


In [4]:
data = np.array(pd.read_csv("energiekalpib_30Torr.csv",skiprows=15,delimiter="\t")).T

In [5]:
plt.errorbar(data[0],data[1],np.sqrt(data[1]))
plt.show()

## Händische schätzung für die Peaks

    - 1: x=250, y=180
    - 2: x=320, y=266
    - 3: x=373, y=158
    - 4: x=551, y=159
    
Mulitgauss mit den Werten konfigurieren und Fehler berechnen

In [6]:
err = np.sqrt(data[1])
err = np.where(err == 0, 1, err)

In [29]:
y = data[1]
x = data[0]


g_1 = GaussianModel(prefix="g_1")
pars = g_1.guess(y,x=x)

pars['g_1center'].set(250, min=240, max=275)
pars['g_1sigma'].set(40, min=33)
pars['g_1amplitude'].set(20000, min=15000)

g_2 = GaussianModel(prefix="g_2")
pars.update( g_2.make_params())

pars['g_2center'].set(320, min=300, max=345)
pars['g_2sigma'].set(15, min=3)
pars['g_2amplitude'].set(2600, min=200)



g_3 = GaussianModel(prefix="g_3")
pars.update( g_3.make_params())

pars['g_3center'].set(373, min=350, max=400)
pars['g_3sigma'].set(15, min=3)
pars['g_3amplitude'].set(158, min=10)


g_4 = GaussianModel(prefix="g_4")
pars.update( g_4.make_params())

pars['g_4center'].set(551, min=530, max=570)
pars['g_4sigma'].set(15, min=3)
pars['g_4amplitude'].set(159, min=10)


mod = g_1 + g_2 + g_3 + g_4

out = mod.fit(y,pars,x=x,weights=1/err)

In [30]:
print(out.fit_report())
out.plot_fit()
plt.show()

[[Model]]
    (((Model(gaussian, prefix='g_1') + Model(gaussian, prefix='g_2')) + Model(gaussian, prefix='g_3')) + Model(gaussian, prefix='g_4'))
[[Fit Statistics]]
    # function evals   = 834
    # data points      = 1023
    # variables        = 12
    chi-square         = 2029.989
    reduced chi-square = 2.008
[[Variables]]
    g_1sigma:       38.6118037 +/- 0        (0.00%) (init= 40)
    g_1center:      275        +/- 0        (0.00%) (init= 250)
    g_1amplitude:   15000      +/- 0        (0.00%) (init= 20000)
    g_1fwhm:        90.9238476 +/- 0        (0.00%)  == '2.3548200*g_1sigma'
    g_2center:      319.666990 +/- 0        (0.00%) (init= 320)
    g_2amplitude:   6421.91306 +/- 0        (0.00%) (init= 2600)
    g_2sigma:       14.3413405 +/- 0        (0.00%) (init= 15)
    g_2fwhm:        33.7712755 +/- 0        (0.00%)  == '2.3548200*g_2sigma'
    g_3center:      371.338095 +/- 0        (0.00%) (init= 373)
    g_3sigma:       19.2417973 +/- 0        (0.00%) (init= 15)
   